In [1]:
# Parameters
day_obs = '2024-06-26'
instrument = 'latiss'

<a class="anchor" id="imports"></a>
## Imports and General Setup

In [2]:
import os
from lsst.summit.utils import ConsDbClient
import requests
import pandas as pd
from IPython.display import display, Markdown, display_markdown

# URL = "https://usdf-rsp.slac.stanford.edu/consdb/" 
# Need to add that part about the headers to client flow through
URL = "http://consdb-pq.consdb:8080/consdb" # Don't use this one

os.environ["no_proxy"] += ",.consdb"

access_token = os.getenv("ACCESS_TOKEN")
headers = {"Authorization": f"Bearer {access_token}"}

# This is how the session object should access the ACCESS Token from the headers
#sesh = requests.Session()
#sesh.headers.update(headers)

%matplotlib inline

In [7]:
from lsst.summit.utils import ConsDbClient
client = ConsDbClient(URL)
print(client)
#import sqlalchemy
#connection = sqlalchemy.create_engine('postgresql://usdf@usdf-summitdb.slac.stanford.edu/exposurelog')
#print(connection)
display_markdown('### Consolidated Database is accessible',raw=True)

### Consolidated Database is accessible

<a class="anchor" id="get_records"></a>
## Get Records

In [8]:
day_obs_int = int(day_obs.replace('-', ''))

visit_query1 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

instrument = 'lsstcomcam'
visit_query2 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

instrument = 'lsstcomcamsim'
visit_query3 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

#what is a quicklook
quicklook_query = f'''
    SELECT q.*  FROM cdb_{instrument}.visit1_quicklook as q,
    cdb_{instrument}.visit1 as v
     WHERE v.day_obs = {day_obs_int} and q.visit_id = v.visit_id
'''

# Potentially print some schema information for debugging
try:
    print(client.schema())  # list the instruments
    print(client.schema('latiss'))  # list tables for an instrument
    print(client.schema('latiss', 'cdb_latiss.exposure_flexdata')) # specifically flexdata table
    
except requests.HTTPError or requests.JSONDecodeError:
    print(client.schema())  # list the instruments
    print(client.schema('latiss'))  # list tables for an instrument
    print(client.schema('latiss', 'cdb_latiss.exposure_flexdata'))

try:
    visits_latiss = client.query(visit_query1).to_pandas()
    visits_lsstcc = client.query(visit_query2).to_pandas()
    visits_lsstccs = client.query(visit_query3).to_pandas()

except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    visits_latiss = client.query(visit_query1).to_pandas()
    visits_lsstcc = client.query(visit_query2).to_pandas()
    visits_lsstccs = client.query(visit_query3).to_pandas()

quicklook = client.query(quicklook_query).to_pandas()

# Assumes at the USDF
#visits_latiss_try = pd.read_sql(visit_query1, connection)
#quicklook_try = pd.read_sql(quicklook_query, connection)

if len(visits_latiss) > 0:
    print(f"Retrieved {len(visits_latiss)} visits from consdb")
    obj_vis = len(visits_latiss.query('img_type == "OBJECT"'))
    print(f"About {obj_vis} of these are object images")

if len(quicklook) > 0:
    visits_latiss = visits_latiss.join(quicklook, on='visit_id', lsuffix='', rsuffix='_q')
    print(f"And added quicklook stats")


['latiss', 'lsstcam', 'lsstcamsim', 'lsstcomcam', 'lsstcomcamsim']
['cdb_latiss.ccdvisit1_quicklook', 'cdb_latiss.ccdexposure', 'cdb_latiss.exposure', 'cdb_latiss.exposure_flexdata', 'cdb_latiss.exposure_flexdata_schema', 'cdb_latiss.visit1_quicklook', 'cdb_latiss.ccdexposure_flexdata', 'cdb_latiss.ccdexposure_flexdata_schema', 'cdb_latiss.ccdexposure_camera']
{'obs_id': ('BIGINT', 'None'), 'key': ('VARCHAR(128)', 'None'), 'value': ('TEXT', 'None')}
Retrieved 786 visits from consdb
About 0 of these are object images
And added quicklook stats


## The schemas in cdb are
`latiss`, `lsstcomcam`, `lsstcomcamsim`

## And each have the following tables
`exposure`, `visit1`

`ccdexposure`, `ccdvisit1`

`ccdexposure_camera`

`visit1_quicklook`, `ccdvisit1_quicklook`

 `exposure_flexdata`, `ccdexposure_flexdata`

 ## We can find more info on the schemas at https://sdm-schemas.lsst.io/

<a class="anchor" id="table"></a>
## Tables of (mostly raw) results

In [5]:
data = pd.DataFrame(visits_latiss)
print(f"Latiss has {len(data.columns)} columns")
print(f"They are {data.columns}")

Latiss has 138 columns
They are Index(['visit_id', 'exposure_name', 'controller', 'day_obs', 'seq_num',
       'physical_filter', 'band', 's_ra', 's_dec', 'sky_rotation',
       ...
       'low_snr_source_count_median', 'low_snr_source_count_total',
       'high_snr_source_count_min', 'high_snr_source_count_max',
       'high_snr_source_count_median', 'high_snr_source_count_total',
       'seeing_zenith_500nm_min', 'seeing_zenith_500nm_max',
       'seeing_zenith_500nm_median', 'n_inputs'],
      dtype='object', length=138)


In [6]:
data

,visit_id,exposure_name,controller,day_obs,seq_num,physical_filter,band,s_ra,s_dec,sky_rotation,...,low_snr_source_count_median,low_snr_source_count_total,high_snr_source_count_min,high_snr_source_count_max,high_snr_source_count_median,high_snr_source_count_total,seeing_zenith_500nm_min,seeing_zenith_500nm_max,seeing_zenith_500nm_median,n_inputs
0,2024062600001,AT_O_20240626_000001,O,20240626,1,empty~blue300lpmm_qn1,EMPTY,0.000000,0.000000,100.121772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024062600002,AT_O_20240626_000002,O,20240626,2,empty~empty,EMPTY,0.000000,0.000000,100.121772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024062600003,AT_O_20240626_000003,O,20240626,3,empty~empty,EMPTY,143.759651,4.453860,172.765556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024062600004,AT_O_20240626_000004,O,20240626,4,SDSSg_65mm~blue300lpmm_qn1,g,114.553192,-20.189471,100.004462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024062600005,AT_O_20240626_000005,O,20240626,5,SDSSr_65mm~empty,r,0.000000,0.000000,2.557083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2024062600796,AT_O_20240626_000796,O,20240626,796,empty~holo4_003,EMPTY,307.075000,-87.472194,146.771807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796,2024062600797,AT_O_20240626_000797,O,20240626,797,empty~holo4_003,EMPTY,307.075000,-87.472194,146.771489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
797,2024062600798,AT_O_20240626_000798,O,20240626,798,empty~blue300lpmm_qn1,EMPTY,307.075000,-87.472194,146.772684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,2024062600799,AT_O_20240626_000799,O,20240626,799,empty~blue300lpmm_qn1,EMPTY,307.075000,-87.472194,146.770220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.DataFrame(visits_lsstcc)
#Yes this is empty

In [ ]:
pd.DataFrame(visits_lsstccs)

# Rapid analysis was mentioned
ccdvisit1_quicklook holds some psf* attributes
quicklook might mean rapid analysis

In [ ]:
print(f"Many Quicklook columns: {len(quicklook.columns)}")
print(f"They include: {quicklook.columns}")

In [ ]:
quicklook

In [ ]:
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
service_loc = os.environ.get('EXTERNAL_INSTANCE_URL', usdf)
DDV = f"{service_loc}/rubintv-dev/ddv/index.html"
display_markdown('## Access DDV part of RubinTV', raw=True)
DDV